In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
import seaborn as sns

In [ ]:
## Load everything
filename = 'Source_cats/LAE_800deg_z2.5-3.2/'

pm_data = np.load(filename + 'pm_flx.npy')
pm_data_noerr = np.load(filename + 'pm_flx_no_line_no_err.npy')

mock = {}

aux = []
with open(filename + 'SEDs.csv') as csv_file:
    rdr = csv.reader(csv_file, delimiter=',')
    for row in rdr:
        aux.append(row)
mock['SEDs'] = np.array(aux, dtype=float)
aux = []
with open(filename + 'SEDs_no_line.csv') as csv_file:
    rdr = csv.reader(csv_file, delimiter=',')
    for row in rdr:
        aux.append(row)
mock['SEDs_no_line'] = np.array(aux, dtype=float)

utils = np.load(filename + 'utils.npy', allow_pickle=True).item()

mock['redshift_Lya_Arr'] = utils['z_Arr']
mock['w_Arr'] = utils['w_Arr']
mock['EW_Arr'] = utils['EW_Arr']

## Somehow, some bright mocks are sneaking in. So, get rid of them.
mask_bright = np.where(pm_data_noerr[-3] <= 1e-17)[0]
pm_data_noerr = pm_data_noerr[:, mask_bright]
pm_data = pm_data[:, mask_bright]
for key in mock.keys():
    if key == 'w_Arr': continue
    mock[key] = mock[key][mask_bright]

err_fit_params = np.load('npy/err_fit_params_minijpas.npy')
m = err_fit_params[:, 0].reshape(-1, 1)
b = err_fit_params[:, 1].reshape(-1, 1)
pm_err = np.load(filename + 'pm_flx_err.npy')[:, mask_bright]

w_central = central_wavelength()

bb_fwhm = [
    nb_fwhm(-4),
    nb_fwhm(-3),
    nb_fwhm(-2),
    nb_fwhm(-1)
]

In [ ]:
mask = flux_to_mag(pm_data[-2], w_central[-2]) < 23

In [ ]:
pm_data = pm_data[:, mask]
pm_err = pm_err[:, mask]

In [ ]:
N_sources = pm_data.shape[1]
zspec = mock['redshift_Lya_Arr'][mask]
mag = flux_to_mag(pm_data, w_central[-2])

In [ ]:
n_bins = 35
n_steps_ew = 8
ew_min = 0
ew_max = 70
mag_min = 20
mag_max = 27
z_min = 2.3
z_max = 3.5
comp_mat_z = np.zeros((n_steps_ew, n_bins - 1))
puri_mat_z = np.copy(comp_mat_z)
puri_mat_rsdss = np.copy(comp_mat_z)
comp_mat_rsdss = np.copy(comp_mat_z)
detec_r = []
detec_z = []
for i, ew0min in enumerate(np.linspace(ew_min, ew_max, n_steps_ew)):
    print('EW0 = {}...'.format(int(ew0min)))
    sel, sel_single, list_lya, _ = QSO_find_lines(
        pm_data, pm_err, 6, 50, ew0min, 15, 6
    )

    sel_mask = np.zeros(N_sources).astype(bool)
    sel_mask[sel_single] = True

    z_nb_Arr = np.ones(N_sources) * -1
    for src in sel_single:
        z_nb_Arr[src] = w_central[list_lya[src]] / 1215.67 - 1

    # Histograms as a function of z
    bins = np.linspace(z_min, z_max, n_bins)
    h_zspec, b = np.histogram(zspec, bins=bins)
    h_detec_z, _ = np.histogram(zspec[sel_mask], bins=bins)
    h_detec_r_z, _ = np.histogram(zspec[sel_mask & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)

    # Histograms as a function of rSDSS
    bins = np.linspace(mag_min, mag_max, n_bins)
    h_rsdss, b = np.histogram(mag[-2], bins=bins)
    h_detec_rsdss, _ = np.histogram(mag[-2, sel_mask], bins=bins)
    h_detec_r_rsdss, _ = np.histogram(mag[-2, sel_mask & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)
    
    comp_mat_rsdss[i] = h_detec_r_rsdss / h_rsdss
    puri_mat_rsdss[i] = h_detec_r_rsdss / h_detec_rsdss
    comp_mat_z[i] = h_detec_r_z / h_zspec
    puri_mat_z[i] = h_detec_r_z / h_detec_z

    detec_z.append(h_detec_z)
    detec_r.append(h_detec_rsdss)
detec_z = np.array(detec_z)
detec_r = np.array(detec_r)

In [ ]:
mock['EW_Arr']

In [ ]:
# COMPLETENESS z
fig, (ax0, ax1)= plt.subplots(1, 2, figsize=(10, 5),
    gridspec_kw={'width_ratios': [4, 1]})
plt.subplots_adjust(wspace=-0.1)

xticks = ['{0:0.2f}'.format(n) for n in np.linspace(z_min, z_max, n_bins - 1)]
yticks = ['{0:0.0f}'.format(n) for n in np.linspace(ew_min, ew_max, n_steps_ew)]

comp_mat_z[np.isnan(comp_mat_z)] = 0
sns.heatmap(comp_mat_z, vmin=0, vmax=1, ax=ax0)
ax0.set_xticklabels(xticks, rotation='vertical')
ax0.set_yticklabels(yticks, rotation='horizontal')
ax0.set_xlabel('Redshift', fontsize=15)
ax0.set_ylabel('Minimum EW$_0$', fontsize=15)
ax0.set_title('Completeness (z)', fontsize=20)

ax1.plot(np.sum(comp_mat_z * h_zspec, axis=1) / np.sum(h_zspec),
    np.linspace(ew_min, ew_max, n_steps_ew))
ax1.set_ylabel('Minimum EW$_0$', fontsize=15)
ax1.invert_yaxis()
ax1.set_xlabel('Total completeness', fontsize=15)
ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
plt.show()

# PURITY z
fig, (ax0, ax1)= plt.subplots(1, 2, figsize=(10, 5),
    gridspec_kw={'width_ratios': [4, 1]})
plt.subplots_adjust(wspace=-0.1)

xticks = ['{0:0.2f}'.format(n) for n in np.linspace(z_min, z_max, n_bins - 1)]
yticks = ['{0:0.0f}'.format(n) for n in np.linspace(ew_min, ew_max, n_steps_ew)]

puri_mat_z[np.isnan(puri_mat_z)] = 0
sns.heatmap(puri_mat_z, vmin=0, vmax=1, ax=ax0)
ax0.set_xticklabels(xticks, rotation='vertical')
ax0.set_yticklabels(yticks, rotation='horizontal')
ax0.set_xlabel('Redshift', fontsize=15)
ax0.set_ylabel('Minimum EW$_0$', fontsize=15)
ax0.set_title('Purity (z)', fontsize=20)

ax1.plot(np.sum(puri_mat_z * detec_z, axis=1) / np.sum(detec_z, axis=1),
    np.linspace(ew_min, ew_max, n_steps_ew))
ax1.set_ylabel('Minimum EW$_0$', fontsize=15)
ax1.invert_yaxis()
ax1.set_xlabel('Total Purity', fontsize=15)
ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
plt.show()

# COMPLETENESS r
fig, (ax0, ax1)= plt.subplots(1, 2, figsize=(10, 5),
    gridspec_kw={'width_ratios': [4, 1]})
plt.subplots_adjust(wspace=-0.1)

xticks = ['{0:0.2f}'.format(n) for n in np.linspace(mag_min, mag_max, n_bins - 1)]
yticks = ['{0:0.0f}'.format(n) for n in np.linspace(ew_min, ew_max, n_steps_ew)]

comp_mat_rsdss[np.isnan(comp_mat_rsdss)] = 0
sns.heatmap(comp_mat_rsdss, vmin=0, vmax=1, ax=ax0)
ax0.set_xticklabels(xticks, rotation='vertical')
ax0.set_yticklabels(yticks, rotation='horizontal')
ax0.set_xlabel('r', fontsize=15)
ax0.set_ylabel('Minimum EW$_0$', fontsize=15)
ax0.set_title('Completeness (rSDSS)', fontsize=20)

ax1.plot(np.sum(comp_mat_rsdss * h_rsdss, axis=1) / np.sum(h_rsdss),
    np.linspace(ew_min, ew_max, n_steps_ew))
ax1.set_ylabel('Minimum EW$_0$', fontsize=15)
ax1.invert_yaxis()
ax1.set_xlabel('Total completeness', fontsize=15)
ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
plt.show()

# PURITY r
fig, (ax0, ax1)= plt.subplots(1, 2, figsize=(10, 5),
    gridspec_kw={'width_ratios': [4, 1]})
plt.subplots_adjust(wspace=-0.1)

xticks = ['{0:0.2f}'.format(n) for n in np.linspace(mag_min, mag_max, n_bins - 1)]
yticks = ['{0:0.0f}'.format(n) for n in np.linspace(ew_min, ew_max, n_steps_ew)]

puri_mat_rsdss[np.isnan(puri_mat_rsdss)] = 0
sns.heatmap(puri_mat_rsdss, vmin=0, vmax=1, ax=ax0)
ax0.set_xticklabels(xticks, rotation='vertical')
ax0.set_yticklabels(yticks, rotation='horizontal')
ax0.set_xlabel('r', fontsize=15)
ax0.set_ylabel('Minimum EW$_0$', fontsize=15)
ax0.set_title('Purity (rSDSS)', fontsize=20)

ax1.plot(np.sum(puri_mat_rsdss * detec_r, axis=1) / np.sum(detec_r, axis=1),
    np.linspace(ew_min, ew_max, n_steps_ew))
ax1.set_ylabel('Minimum EW$_0$', fontsize=15)
ax1.invert_yaxis()
ax1.set_xlabel('Total Purity', fontsize=15)
ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(detec_r, ax=ax)
ax.set_xticklabels(xticks, rotation='vertical')
ax.set_yticklabels(yticks, rotation='horizontal')
ax.set_xlabel('rSDSS', fontsize=15)
ax.set_ylabel('EW$_0$ min ($\AA$)', fontsize=15)
plt.show()

fig, ax = plt.subplots(figsize=(8, 6))

xticks = ['{0:0.2f}'.format(n) for n in np.linspace(z_min, z_max, n_bins - 1)]
yticks = ['{0:0.0f}'.format(n) for n in np.linspace(ew_min, ew_max, n_steps_ew)]

sns.heatmap(detec_z, ax=ax)
ax.set_xticklabels(xticks, rotation='vertical')
ax.set_yticklabels(yticks, rotation='horizontal')
ax.set_xlabel('z', fontsize=15)
ax.set_ylabel('EW$_0$ min ($\AA$)', fontsize=15)
plt.show()